In [1]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash 
import statsmodels.api as sm
from matplotlib import pyplot as plt

C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load the data from the CSV file (adjust header as necessary)
data = pd.read_csv("Water_Leakages.csv")

# Display the first few rows of the updated DataFrame
data.head()


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,water leakage,SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,water leakage,SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.992790,12.874319,13.007650,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125,NaN
2,Afghanistan,AFG,water leakage,SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,10.086000,12.495000,14.902000,17.303000,17.292000,17.339000,16.053000,NaN,NaN,NaN
3,Africa Western and Central,AFW,water leakage,SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,8.432707,8.860586,8.690286,8.785932,8.910159,9.005565,10.444833,9.874787,9.726148,NaN
4,Angola,AGO,water leakage,SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.428000,16.437000,16.436000,16.363000,16.343000,16.283000,17.967000,17.523000,17.279000,NaN


In [3]:
#shape of data 
data.shape

(266, 68)

In [4]:
data.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       'Unnamed: 67'],
      dtype='object')

In [5]:
#data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 68 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    266 non-null    object 
 1   Country Code    266 non-null    object 
 2   Indicator Name  266 non-null    object 
 3   Indicator Code  266 non-null    object 
 4   1960            0 non-null      float64
 5   1961            0 non-null      float64
 6   1962            0 non-null      float64
 7   1963            0 non-null      float64
 8   1964            0 non-null      float64
 9   1965            0 non-null      float64
 10  1966            0 non-null      float64
 11  1967            0 non-null      float64
 12  1968            0 non-null      float64
 13  1969            0 non-null      float64
 14  1970            0 non-null      float64
 15  1971            0 non-null      float64
 16  1972            0 non-null      float64
 17  1973            0 non-null      flo

In [6]:
#finding if there are missing data 
data.isnull().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              266
                 ... 
2019               31
2020               31
2021               33
2022               37
Unnamed: 67       266
Length: 68, dtype: int64

In [7]:
#findingmissing value percentage 
data.isnull().sum()/data.shape[0]*100

Country Name        0.000000
Country Code        0.000000
Indicator Name      0.000000
Indicator Code      0.000000
1960              100.000000
                     ...    
2019               11.654135
2020               11.654135
2021               12.406015
2022               13.909774
Unnamed: 67       100.000000
Length: 68, dtype: float64

In [8]:
#finding the duplicates 
data.duplicated().sum()

0

In [9]:
data.head(1)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,water leakage,SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can observe that no data was recorded from 1960 to 1990, as there is only one unique value—a blank space—for nearly all, if not all, of the years in this range. Therefore, it is safe to conclude that these years should be dropped from our dataset.

In [10]:
from plotly.subplots import make_subplots  # Import make_subplots
if data.isnull().values.any():
    missing_percentage = (data.isnull().sum() / len(data)) * 100
    columns_with_missing = missing_percentage[missing_percentage > 0]
    
    # Separate columns with 50% or greater missing values and those with less than 50%
    high_missing = columns_with_missing[columns_with_missing >= 50]
    low_missing = columns_with_missing[columns_with_missing < 50]
    
    # Creating subplots
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "pie"}, {"type": "pie"}]]
    )
    
    # Donut plot for columns with 50% or greater missing values
    fig.add_trace(
        go.Pie(
            values=high_missing.values,
            labels=high_missing.index,
            hole=.3,
            textinfo='percent+label'
        ), 
        row=1, col=1
    )
    
    # Donut plot for columns with less than 50% missing values
    fig.add_trace(
        go.Pie(
            values=low_missing.values,
            labels=low_missing.index,
            hole=.3,
            textinfo='percent+label'
        ), 
        row=1, col=2
    )
    
    fig.update_layout(
        title_text="Percentage of Missing Values by Column",  
        width=1350,
        height=600,
        annotations=[
            dict(
                text="Columns with ≥50% Missing Values",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.25, y=-0.15,
                xanchor="center", yanchor="top",
                font=dict(size=14)
            ),
            dict(
                text="Columns with <50% Missing Values",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.75, y=-0.15,
                xanchor="center", yanchor="top",
                font=dict(size=14)
            )
        ]
    )
    
    fig.show()
else:
    print("No missing values in the dataset.")

We can observed that from 1960 to 1990 no data was acquired, there are no obeservations at all in the years in this range. Therefore With the knowledge that all columns with more than 50% missing data are to be dropped since any imputions would be regarded as in accurate.

In [11]:
#droping the cols 
df1= data.drop(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007',], axis= 1 ) 

In [12]:
df1.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,water leakage,SL.UEM.1524.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,water leakage,SL.UEM.1524.ZS,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.99279,12.874319,13.00765,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125,NaN


Understanding that the data represents Waker leakage by Percentage, measured as a percentage, we can now remove the two columns, Indicator Name and Indicator Code

In [13]:
df = df1.drop(columns=['Indicator Name', 'Indicator Code'])

In [14]:
df.dtypes

Country Name     object
Country Code     object
2008            float64
2009            float64
2010            float64
2011            float64
2012            float64
2013            float64
2014            float64
2015            float64
2016            float64
2017            float64
2018            float64
2019            float64
2020            float64
2021            float64
2022            float64
Unnamed: 67     float64
dtype: object

In [15]:
# Delete the last column from the dataset
df = df.iloc[:, :-1]

# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Display the first few rows of the updated DataFrame
df.head()

,Country Name,Country Code,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.992790,12.874319,13.007650,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
2,Afghanistan,AFG,10.852000,10.794000,10.667000,10.441000,10.393000,10.224000,10.086000,12.495000,14.902000,17.303000,17.292000,17.339000,16.053000,NaN,NaN
3,Africa Western and Central,AFW,8.528799,8.588388,8.626529,8.531824,8.539699,8.084479,8.432707,8.860586,8.690286,8.785932,8.910159,9.005565,10.444833,9.874787,9.726148
4,Angola,AGO,8.641000,8.659000,19.800000,17.216000,16.791000,16.595000,16.428000,16.437000,16.436000,16.363000,16.343000,16.283000,17.967000,17.523000,17.279000


In [16]:
# Extract the columns to check first, all but "Country Name" and "Country Code"
cols_2_see = df.columns.difference(['Country Name', 'Country Code'])

NaNs = df[cols_2_see].isna().all(axis=1)
df[NaNs]

,Country Name,Country Code,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Antigua and Barbuda,ATG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Bermuda,BMU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Curacao,CUW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,Cayman Islands,CYM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,Dominica,DMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,Faroe Islands,FRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,"Micronesia, Fed. Sts.",FSM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Remove these rows from the dataset
df = df[~NaNs]
df.head(2)

,Country Name,Country Code,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Africa Eastern and Southern,AFE,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.99279,12.874319,13.00765,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
2,Afghanistan,AFG,10.852000,10.794000,10.667000,10.441000,10.393000,10.224000,10.08600,12.495000,14.90200,17.303000,17.292000,17.339000,16.053000,NaN,NaN


In [18]:
df.shape

(235, 17)

In [19]:
#importing the continents dataset 
data2=pd.read_csv("continents-according-to-our-world-in-data.csv")  
data2.sample(n=5)

,Entity,Code,Year,Continent
193,Parma,OWID_PMA,2015,Europe
233,South Korea,KOR,2015,Asia
245,Tajikistan,TJK,2015,Asia
185,Norway,NOR,2015,Europe
241,Sweden,SWE,2015,Europe


In [20]:
# Merge the DataFrames based on 'Country Name' and 'Country Code'
df_merged = df.merge(data2[['Entity', 'Continent', 'Code']],
                     left_on=['Country Name', 'Country Code'],
                     right_on=['Entity', 'Code'],
                     how='left')

# Move the 'Continent' column to position 3
continent_col = df_merged.pop('Continent')
df_merged.insert(2, continent_col.name, continent_col)

# Drop the redundant columns 'Entity' and 'Code'
df_merged = df_merged.drop(['Entity', 'Code'], axis=1)

# Display the first few rows of the updated DataFrame
df_merged.head(2)

,Country Name,Country Code,Continent,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Africa Eastern and Southern,AFE,NaN,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.99279,12.874319,13.00765,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
1,Afghanistan,AFG,Asia,10.852000,10.794000,10.667000,10.441000,10.393000,10.224000,10.08600,12.495000,14.90200,17.303000,17.292000,17.339000,16.053000,NaN,NaN


In [21]:
# Select the columns from 1991 to 2022
columns_to_check = df_merged.loc[:, '2008':'2022']

# Find columns with NaN values
columns_with_na = columns_to_check.columns[columns_to_check.isnull().any()]

# Display the columns with NaN values
print("Columns with NaN values from 1991 to 2022:")
print(columns_with_na)

Columns with NaN values from 1991 to 2022:
Index(['2021', '2022'], dtype='object')


In [22]:
#Filter rows where '2021' or '2022' have NaN values
nan_rows = df_merged[df_merged[['2021', '2022']].isna().any(axis=1)]

# Select the 'Country Name' column along with '2021' and '2022' columns
result = nan_rows[['Country Name', '2021', '2022']]

result

,Country Name,2021,2022
1,Afghanistan,NaN,NaN
56,Europe & Central Asia (excluding high income),17.445115,NaN
139,Myanmar,NaN,NaN
178,Russian Federation,16.144000,NaN
203,Europe & Central Asia (IDA & IBRD countries),17.237121,NaN
219,Ukraine,19.110000,NaN


In [23]:
# Updating the values for Afghanistan and the Russian Federation in the df_merged DataFrame
df_merged.loc[df_merged['Country Name'] == 'Afghanistan', '2021'] = 16.39
df_merged.loc[df_merged['Country Name'] == 'Afghanistan', '2022'] = 17.75
df_merged.loc[df_merged['Country Name'] == 'Russian Federation', '2022'] = 14.37

# Verify the changes for Afghanistan and the Russian Federation
print(df_merged[df_merged['Country Name'].isin(['Afghanistan', 'Russian Federation'])][['Country Name', '2021', '2022']])

           Country Name    2021   2022
1           Afghanistan  16.390  17.75
178  Russian Federation  16.144  14.37


In [24]:
# Fill NaN values with the mean of each column
df_merged[['2021', '2022']] = df_merged[['2021', '2022']].apply(lambda col: col.fillna(col.mean()))

In [25]:
df_merged.head(3)

,Country Name,Country Code,Continent,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Africa Eastern and Southern,AFE,NaN,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.992790,12.874319,13.007650,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
1,Afghanistan,AFG,Asia,10.852000,10.794000,10.667000,10.441000,10.393000,10.224000,10.086000,12.495000,14.902000,17.303000,17.292000,17.339000,16.053000,16.390000,17.750000
2,Africa Western and Central,AFW,NaN,8.528799,8.588388,8.626529,8.531824,8.539699,8.084479,8.432707,8.860586,8.690286,8.785932,8.910159,9.005565,10.444833,9.874787,9.726148


In [26]:
# Get the sum of na values in the continent col 
df_merged['Continent'].isna().sum()

75

In [27]:
# Filter rows where 'Continent' has NaN values
rows_with_na = df_merged[df_merged['Continent'].isna()]
rows_with_na

,Country Name,Country Code,Continent,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Africa Eastern and Southern,AFE,NaN,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.992790,12.874319,13.007650,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
2,Africa Western and Central,AFW,NaN,8.528799,8.588388,8.626529,8.531824,8.539699,8.084479,8.432707,8.860586,8.690286,8.785932,8.910159,9.005565,10.444833,9.874787,9.726148
5,Arab World,ARB,NaN,23.806125,22.609268,23.203179,25.307581,26.960191,27.061260,27.068250,28.245065,27.876574,28.554279,27.544687,26.373932,28.255717,27.276824,26.682374
19,"Bahamas, The",BHS,NaN,17.155000,26.255000,26.688000,27.287000,30.740000,34.847000,29.238000,25.001000,25.944000,21.561000,24.250000,24.434000,29.898000,27.210000,23.854000
26,Brunei Darussalam,BRN,NaN,19.285000,20.092000,21.143000,22.095000,22.820000,23.546000,24.397000,26.258000,27.812000,29.652000,31.453000,21.513000,27.129000,24.265000,23.887000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,"Venezuela, RB",VEN,NaN,12.365000,12.135000,14.176000,14.037000,14.077000,16.221000,14.823000,13.235000,11.673000,12.336000,12.446000,12.712000,13.800000,13.445000,11.388000
226,Virgin Islands (U.S.),VIR,NaN,20.919000,24.076000,24.068000,25.339000,26.018000,27.559000,27.615000,26.595000,27.849000,26.931000,26.988000,27.412000,29.674000,28.567000,27.198000
227,Viet Nam,VNM,NaN,5.317000,4.494000,3.474000,3.265000,3.703000,4.632000,4.756000,6.351000,6.788000,7.016000,4.505000,5.501000,6.422000,6.805000,7.438000
229,World,WLD,NaN,13.232263,14.253475,14.329581,14.431704,14.704453,14.978708,15.081434,15.557598,15.860100,15.959626,15.919090,15.236170,18.401152,16.356035,15.581516


Here we notice that there are 75 rows, or rather 75 countries that are not 'continent classifies'. Maybe the issue is with the naming of the countries, that is to say 'South Africa', 'Suid Afrika' and 'South Africa ' are all different with the same country code. So we try matching using the country code alone

In [28]:
# Merge the DataFrames based only on 'Country Code'
merged_data1 = df.merge(data2[['Code', 'Continent']],
                              left_on='Country Code',
                              right_on='Code',
                              how='left')

# Drop the redundant column 'Code'
merged_data1 = merged_data1.drop(['Code'], axis=1)

# Move column "Continent" to position 3
col = merged_data1.pop('Continent')
merged_data1.insert(2, col.name, col)

merged_data1['Continent'].isna().sum()

49

In [29]:
rows_with_na.shape

(75, 18)

In [30]:
# Check if the column 'Continent' exists and has NaN values
countries_without_continent = df_merged[df_merged['Continent'].isna()]

# Display the countries that do not have a continent assigned
print("Countries without a continent assigned:")
print(countries_without_continent[['Country Name', 'Continent','Country Code']])

Countries without a continent assigned:
                    Country Name Continent Country Code
0    Africa Eastern and Southern       NaN          AFE
2     Africa Western and Central       NaN          AFW
5                     Arab World       NaN          ARB
19                  Bahamas, The       NaN          BHS
26             Brunei Darussalam       NaN          BRN
..                           ...       ...          ...
225                Venezuela, RB       NaN          VEN
226        Virgin Islands (U.S.)       NaN          VIR
227                     Viet Nam       NaN          VNM
229                        World       NaN          WLD
231                  Yemen, Rep.       NaN          YEM

[75 rows x 3 columns]


In [31]:
# Display the list of country codes without continents  
print("Country codes of countries without a continent assigned:")
print(countries_without_continent['Country Code'].tolist())

Country codes of countries without a continent assigned:
['AFE', 'AFW', 'ARB', 'BHS', 'BRN', 'CEB', 'CHI', 'COD', 'COG', 'CPV', 'CSS', 'EAP', 'EAR', 'EAS', 'ECA', 'ECS', 'EGY', 'EMU', 'EUU', 'FCS', 'GMB', 'HIC', 'HKG', 'HPC', 'IBD', 'IBT', 'IDA', 'IDB', 'IDX', 'IRN', 'KGZ', 'KOR', 'LAC', 'LAO', 'LCA', 'LCN', 'LDC', 'LIC', 'LMC', 'LMY', 'LTE', 'MAC', 'MEA', 'MIC', 'MNA', 'NAC', 'OED', 'OSS', 'PRE', 'PRK', 'PSE', 'PSS', 'PST', 'RUS', 'SAS', 'SSA', 'SSF', 'SST', 'SVK', 'SYR', 'TEA', 'TEC', 'TLA', 'TLS', 'TMN', 'TSA', 'TSS', 'TUR', 'UMC', 'VCT', 'VEN', 'VIR', 'VNM', 'WLD', 'YEM']


In [32]:
df_merged.head(2)

,Country Name,Country Code,Continent,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Africa Eastern and Southern,AFE,NaN,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.99279,12.874319,13.00765,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
1,Afghanistan,AFG,Asia,10.852000,10.794000,10.667000,10.441000,10.393000,10.224000,10.08600,12.495000,14.90200,17.303000,17.292000,17.339000,16.053000,16.390000,17.750000


In [33]:
# clean dataset 
df4 = pd.read_csv("country_continent.csv")

In [34]:
# Merge df3 with df_merged on the 'Country Code' column
data = pd.merge(df4, df_merged, on='Country Code', how='inner')

# Display the first few rows of the merged DataFrame
data.head()

,Country Code,Country Name_x,Continent_x,Country Name_y,Continent_y,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AFE,Africa Eastern and Southern,Africa,Africa Eastern and Southern,NaN,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.992790,12.874319,13.007650,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
1,AFG,Afghanistan,Asia,Afghanistan,Asia,10.852000,10.794000,10.667000,10.441000,10.393000,10.224000,10.086000,12.495000,14.902000,17.303000,17.292000,17.339000,16.053000,16.390000,17.750000
2,AFW,Africa Western and Central,Africa,Africa Western and Central,NaN,8.528799,8.588388,8.626529,8.531824,8.539699,8.084479,8.432707,8.860586,8.690286,8.785932,8.910159,9.005565,10.444833,9.874787,9.726148
3,AGO,Angola,Africa,Angola,Africa,8.641000,8.659000,19.800000,17.216000,16.791000,16.595000,16.428000,16.437000,16.436000,16.363000,16.343000,16.283000,17.967000,17.523000,17.279000
4,ALB,Albania,Europe,Albania,Europe,26.553000,27.324000,30.885000,22.733000,28.987000,31.318000,39.744000,39.874000,36.172000,31.403000,28.305000,27.161000,30.846000,29.505000,27.788000


we can see that we have redundent rows and columns and so we are going to keep Country Name_y, Continent_y which is from df3 and drop Country Name_x , Continent_x which are coming from the pd.merge.

In [35]:
# Drop the 'Country Name_y' and 'Continent_y' columns from the DataFrame 'data'
data = data.drop(columns=['Country Name_y', 'Continent_y'])

In [36]:
data.head(5)

,Country Code,Country Name_x,Continent_x,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AFE,Africa Eastern and Southern,Africa,12.391843,12.543647,14.277086,13.593977,13.402498,13.102415,12.992790,12.874319,13.007650,12.973653,12.887711,13.281005,14.138644,14.137466,14.134125
1,AFG,Afghanistan,Asia,10.852000,10.794000,10.667000,10.441000,10.393000,10.224000,10.086000,12.495000,14.902000,17.303000,17.292000,17.339000,16.053000,16.390000,17.750000
2,AFW,Africa Western and Central,Africa,8.528799,8.588388,8.626529,8.531824,8.539699,8.084479,8.432707,8.860586,8.690286,8.785932,8.910159,9.005565,10.444833,9.874787,9.726148
3,AGO,Angola,Africa,8.641000,8.659000,19.800000,17.216000,16.791000,16.595000,16.428000,16.437000,16.436000,16.363000,16.343000,16.283000,17.967000,17.523000,17.279000
4,ALB,Albania,Europe,26.553000,27.324000,30.885000,22.733000,28.987000,31.318000,39.744000,39.874000,36.172000,31.403000,28.305000,27.161000,30.846000,29.505000,27.788000


In [37]:
# Transform the DataFrame to long format
data_long = pd.melt(data, id_vars=['Country Code', 'Country Name_x', 'Continent_x'],
                    var_name='Year', value_name='Water Leakage')

# Ensure 'Year' is a string
data_long['Year'] = data_long['Year'].astype(str)

# Filter data to include all countries
filtered_data = data_long

# Plotting the animated choropleth map
fig1 = px.choropleth(
    filtered_data,
    locations='Country Code',
    color='Water Leakage',  # Corrected from 'Water Leakage Rate' to 'Water Leakage'
    hover_name='Country Name_x',  
    animation_frame='Year', 
    color_continuous_scale='Viridis',  
    title='Water Leakage Rate Over Time [2008-2022]',
    projection='natural earth'  
)

# Customize layout and increase size of the map
fig1.update_layout(
    geo=dict(
        showcoastlines=True,
        coastlinecolor="Black",
        showland=True,
        landcolor="LightGray",
        showocean=True,
        oceancolor="LightBlue",
        showlakes=True,
        lakecolor="LightBlue"
    ),
    coloraxis_colorbar_title='Water Leakage Rate',
    title_font_size=24,
    title_font_family='Arial',
    width=1250,  # Increase the width of the map
    height=450   # Increase the height of the map
)

# Show the plot
fig1.show()

c:\ProgramData\anaconda3\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [44]:
# Load the data
data_long = pd.melt(data, id_vars=['Country Code', 'Country Name_x', 'Continent_x'],
                    var_name='Year', value_name='Water Leakage')

# Ensure 'Year' is a string
data_long['Year'] = data_long['Year'].astype(str)

# Filter data for specific countries
selected_countries = [
    'South Africa', 
    'Namibia', 
    'Botswana', 
    'Zimbabwe', 
    'Mozambique'
]

# Filter data to include only selected countries
filtered_data = data_long[data_long['Country Name_x'].isin(selected_countries)]

# Create the animated bar plot
fig2 = px.bar(
    filtered_data,
    x='Country Name_x',  # X-axis represents the countries
    y='Water Leakage',  # Y-axis represents water leakage values
    color='Country Name_x',  # Different colors for different countries
    animation_frame='Year',  # Animation over the years
    animation_group='Country Name_x',  # Group animation by country
    title='Water Leakage by Country (2008-2022)',
    labels={'Country Name_x': 'Country', 'Water Leakage': 'Water Leakage'},
)

# Customize layout with specified height and width
fig2.update_layout(
    xaxis_title='Country',
    yaxis_title='Water Leakage',
    title_font_size=24,
    title_font_family='Arial',
    showlegend=False,  # Hide the legend as colors represent countries already
    width=1250,  # Set the width of the plot
    height=550   # Set the height of the plot
)

# Show the plot
fig2.show()

In [39]:
# Filter data for South Africa only
sa_data = data[data['Country Name_x'] == 'South Africa']

# List of yearly columns for 2008 to 2023
yearly_columns = [str(year) for year in range(2008, 2023)]  # Adjusted to include 2023

# Calculate the average water leakage for South Africa for each year
annual_averages_sa = sa_data[yearly_columns].mean()

# Convert the result to a DataFrame for plotting
trend_data_sa = pd.DataFrame({
    'Year': annual_averages_sa.index,
    'Average Water Leakage': annual_averages_sa.values
})

# Plotting the trend line graph for South Africa's water leakage
fig3 = px.line(trend_data_sa, x='Year', y='Average Water Leakage',
                title='Average Water Leakage for South Africa (2008-2023)',
                labels={'Average Water Leakage': 'Average Water Leakage (%)'},
                markers=True,  # Add markers to the line
                line_shape='spline',  # Use smooth curves
                color_discrete_sequence=['#1f77b4'])  # Custom color

# Customize layout with grid lines
fig3.update_layout(
    xaxis_title='Year',
    yaxis_title='Average Water Leakage (%)',
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(255,255,255,0.8)',
    title_font_size=24,
    title_font_family='Arial',
    xaxis=dict(
        tickmode='linear',
        dtick=1,
        showgrid=True,  # Add grid lines for the x-axis
        gridcolor='LightGray'  # Color of the grid lines
    ),
    yaxis=dict(
        showgrid=True,  # Add grid lines for the y-axis
        gridcolor='LightGray'  # Color of the grid lines
    )
)

# Show the plot
fig3.show()

In [40]:
df5 = pd.read_excel("NC Data.xlsx")

# Sort the DataFrame by 'Agricultural Potential'
northern_cape_df_sorted = df5.sort_values(by='Agricultural Potential', ascending=False)

# Create the bar chart
fig4 = go.Figure(data=[
    go.Bar(name='Agricultural Potential', x=northern_cape_df_sorted['City/Town in the NC'], y=northern_cape_df_sorted['Agricultural Potential']),
    go.Bar(name='Size of Economy', x=northern_cape_df_sorted['City/Town in the NC'], y=northern_cape_df_sorted['Size of Economy'])
])

# Update layout of the figure
fig4.update_layout(
    barmode='group',
    title='Agricultural Potential and Size of Economy of Towns in the Northern Cape'
)

# Show the plot
fig4.show()

In [41]:
# Read data from the Excel file
df = pd.read_excel('Leakage_Costs.xlsx')

df.columns = ['Year', 'Water Leakage (%)', 'Water Leakage Cost (Million R)']

# Create a bar plot for Water Leakage Cost
fig5 = px.bar(
    df,
    x='Year',
    y='Water Leakage Cost (Million R)',
    color='Water Leakage Cost (Million R)', 
    color_continuous_scale='Viridis',
    title='Water Leakage Cost by Year (2008-2022)',
    labels={'Water Leakage Cost (Million R)': 'Water Leakage Cost (Million R)'},
    template='plotly_white'
)

# Update layout for the figure
fig5.update_layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='Water Leakage Cost (Million R)'),
    width=1250,  # Set the height to 1250
    height=500     # Set the width to 550
)

# Show the plot
fig5.show()

In [42]:

# Load the data from the Excel file
file_path = 'Economic_Effect.xlsx'
df = pd.read_excel(file_path)

# Filter the data for the relevant years (2018-2022)
data_filtered = df[df['Year'].isin([2018, 2019, 2020, 2021, 2022])]

# Create the Sunburst Chart
sunburst_fig = px.sunburst(
    data_filtered,
    path=['Year', 'Sector'],  # Define the hierarchy for the sunburst chart
    values='Water Leakage Cost (%)',  # Use 'Water Leakage Cost (%)' as the values
    title='Water Leakage Costs by Sector in South Africa (2018-2022)',
    color='Water Leakage Cost (%)',  # Color by Water Leakage Cost
    color_continuous_scale='Greens',  # Change this to a green color scale
    branchvalues='total',  # Calculate branch values as total
    template='plotly_white'
)

# Update layout for the figure
sunburst_fig.update_layout(
    title_font_size=24,
    title_font_family='Arial',
    height=450,  # Set the height to 450
    width=1250   # Set the width to 1250
)

# Show the plot
sunburst_fig.show()

In [43]:
import warnings
warnings.filterwarnings("ignore")

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

# Create the app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SLATE], suppress_callback_exceptions=True)

# Sidebar layout with full height and centered buttons
sidebar = html.Div(
    [
        html.Hr(),
        html.H6("Investing in Infrastructure: Building a Sustainable Water Future", style={'color': 'white', 'text-align': 'center', 'margin-bottom': '10px'}),
        dbc.Button("Water Analysis", href="/visuals1", id="visuals1-button", color="primary", className="mb-2", style={'border-radius': '25px', 'width': '150px'}),
        dbc.Button("Water Impact", href="/visuals2", id="visuals2-button", color="secondary", className="mb-2", style={'border-radius': '25px', 'width': '150px'}),
        html.H6("Every Drop Saved: Cutting Costs Through Smart Leak Management", style={'color': 'white', 'text-align': 'center', 'margin-bottom': '10px'}),
    ],
    style={
        'width': '15%',
        'position': 'fixed', 
        'top': '0',  
        'right': '0', 
        'height': '100%',  
        'backgroundColor': '#343a40',
        'padding': '10px',
        'z-index': '10',  
        'display': 'flex',
        'flexDirection': 'column',
        'justifyContent': 'center',
        'alignItems': 'center'
    }
)

# Title of the dashboard
title = html.H1("Print('Alpha')", style={
    'text-align': 'center', 
    'position': 'fixed', 
    'top': '10px', 
    'left': '50%', 
    'transform': 'translateX(-50%)', 
    'width': '100%', 
    'z-index': '999'
})

# Content layout with fixed height, adjusted for the sidebar
content = html.Div(id="page-content", style={
    'margin-right': '15%',
    'margin-left': '0',
    'margin-top': '60px', 
    'height': 'calc(100vh - 100px)',  
    'overflowY': 'auto',  
    'padding': '10px',  
    'z-index': '1',
})

# Define the first page visuals with proper spacing and centering
first = dbc.Container([
    dbc.Row(
        dbc.Col(dcc.Graph(figure=fig1, style={'width': '100%', 'height': '400px'}), width=12),
        justify="center",  # Center the graph
        style={'margin-bottom': '40px'}  # Space below fig1
    ),
    dbc.Row(
        dbc.Col(html.Hr(style={'border': '1px solid #ccc', 'margin-bottom': '20px'})),  # Divider
    ),
    dbc.Row(
        dbc.Col(dcc.Graph(figure=fig2, style={'width': '100%', 'height': '500px'}), width=12),
        justify="center",  # Center the graph
        style={'margin-bottom': '40px'}  # Space below fig2
    ),
    dbc.Row(
        dbc.Col(html.Hr(style={'border': '1px solid #ccc', 'margin-bottom': '20px'})),  # Divider
    ),
    dbc.Row(
        dbc.Col(dcc.Graph(figure=fig3, style={'width': '100%', 'height': '450px'}), width=12),
        justify="center",  # Center the graph
        style={'margin-bottom': '40px'}  # Space below fig3
    ),
], fluid=True)

# Define the second page visuals with proper spacing and centering
second = dbc.Container([
    dbc.Row(
        dbc.Col(dcc.Graph(figure=sunburst_fig, style={'width': '100%', 'height': '400px'}), width=12),
        justify="center",  # Center the graph
        style={'margin-bottom': '40px'}  # Space below fig6
    ),
    dbc.Row(
        dbc.Col(html.Hr(style={'border': '1px solid #ccc', 'margin-bottom': '20px'})),  # Divider
    ),
    dbc.Row(
        dbc.Col(dcc.Graph(figure=fig5, style={'width': '100%', 'height': '450px'}), width=12),
        justify="center",  # Center the graph
        style={'margin-bottom': '40px'}  # Space below fig5
    ),
    dbc.Row(
        dbc.Col(html.Hr(style={'border': '1px solid #ccc', 'margin-bottom': '20px'})),  # Divider
    ),
    dbc.Row(
        dbc.Col(dcc.Graph(figure=fig4, style={'width': '100%', 'height': '500px'}), width=12),
        justify="center",  # Center the graph
        style={'margin-bottom': '40px'}  # Space below fig4
    ),
], fluid=True)

# App layout
app.layout = html.Div([
    dcc.Location(id='url'),
    sidebar,
    title,
    content
])

# Callback to control navigation
@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def display_page(pathname):
    if pathname == "/visuals1" or pathname == "/":  
        return first  
    elif pathname == "/visuals2":
        return second  
    else:
        return first  

# Run the app on a different port
if __name__ == "__main__":  
    app.run_server(debug=True, port=8020)


Dash app running on http://127.0.0.1:8020/
